In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set(rc={'figure.figsize':(11, 4)})
plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4
plt.ion()
plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

# Tarefa
O objetivo deste trabalho é implementar e treinar uma arquitetura neural para resolver o problema de Part-of-Speech Tagging em português, utilizando o corpus Mac-Morpho. O problema consiste em predizer, para cada palavra em uma sentença, a sua classe gramatical.

# Leitura do corpus
O seguinte trecho de código faz a leitura do corpus Mac-Morpho, disponível em: http://nilc.icmc.usp.br/macmorpho/. O dataset já vem separado em treino, validação e teste, porém aqui eu concateno todos eles para poder facilitar o pré-processamento dos dados.

In [ ]:
!wget http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
!tar zxvf macmorpho-v3.tgz

--2022-01-22 17:35:46--  http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
Resolving nilc.icmc.usp.br (nilc.icmc.usp.br)... 143.107.183.225
Connecting to nilc.icmc.usp.br (nilc.icmc.usp.br)|143.107.183.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2463485 (2.3M) [application/x-gzip]
Saving to: ‘macmorpho-v3.tgz.2’

macmorpho-v3.tgz.2   54%[=========>          ]   1.27M  23.3KB/s    eta 33s    ^C
macmorpho-dev.txt
macmorpho-test.txt
macmorpho-train.txt


In [ ]:
def read_files(tipo):
    filename = f"macmorpho-{tipo}.txt"
    sentences = []
    with open(filename, "r") as data:
        for line in data:
            linha = line.split()
            sentenca = []
            for word_tag in linha:
                word, tag = word_tag.split("_")
                sentenca.append((word, tag))
            sentences.append(sentenca)

    return sentences

dev, test, train = read_files("dev"), read_files("test"), read_files("train")
dev_len, test_len, train_len = len(dev), len(test), len(train)
data = train + dev + test
data[0]

[('Jersei', 'N'),
 ('atinge', 'V'),
 ('média', 'N'),
 ('de', 'PREP'),
 ('Cr$', 'CUR'),
 ('1,4', 'NUM'),
 ('milhão', 'N'),
 ('na', 'PREP+ART'),
 ('venda', 'N'),
 ('da', 'PREP+ART'),
 ('Pinhal', 'NPROP'),
 ('em', 'PREP'),
 ('São', 'NPROP'),
 ('Paulo', 'NPROP'),
 ('.', 'PU')]

In [ ]:
print(f"Train size: {train_len}\nTest size: {test_len}\nValidation size: {dev_len}")

Train size: 37948
Test size: 9987
Validation size: 1997


Aqui, separo as palavras e as tags nos vetores X e Y. Ambos são vetores 2D, onde cada entrada é uma sentença.

In [ ]:
X = []
Y = []

for sentence in data:
    X_sentence = []
    Y_sentence = []
    for word, tag in sentence:         
        X_sentence.append(word) 
        Y_sentence.append(tag)
        
    X.append(X_sentence)
    Y.append(Y_sentence)

print(f"{X[0]} length: {len(X[0])}")
print(f"{Y[0]} length: {len(Y[0])}")
print(f"X size:{len(X)}\nY size:{len(Y)}")

['Jersei', 'atinge', 'média', 'de', 'Cr$', '1,4', 'milhão', 'na', 'venda', 'da', 'Pinhal', 'em', 'São', 'Paulo', '.'] length: 15
['N', 'V', 'N', 'PREP', 'CUR', 'NUM', 'N', 'PREP+ART', 'N', 'PREP+ART', 'NPROP', 'PREP', 'NPROP', 'NPROP', 'PU'] length: 15
X size:49932
Y size:49932


No total, temos 26 tags.

In [ ]:
tags = set([tag for sentence in Y for tag in sentence])
NUM_TAGS = len(tags)
print(f"Tags: {tags}")
print(f"Number of tags: {NUM_TAGS}")

Tags: {'ART', 'PCP', 'N', 'ADV', 'PREP+ART', 'PREP+PROPESS', 'PREP+PROSUB', 'V', 'ADV-KS', 'PROADJ', 'PROPESS', 'PREP+PRO-KS', 'PDEN', 'ADJ', 'KC', 'PREP+PROADJ', 'CUR', 'PREP', 'KS', 'PU', 'PROSUB', 'PRO-KS', 'NUM', 'PREP+ADV', 'NPROP', 'IN'}
Number of tags: 26


# Pré-processamento
Cada palavra de X e cada tag de Y são transformados em um número inteiro utilizando a classe Tokenizer.

In [ ]:
word_tokenizer = Tokenizer(filters=None)
word_tokenizer.fit_on_texts(X)
X_encoded = word_tokenizer.texts_to_sequences(X)

print(f"Original: {X[0]}")
print(f"Encoded: {X_encoded[0]}")

Original: ['Jersei', 'atinge', 'média', 'de', 'Cr$', '1,4', 'milhão', 'na', 'venda', 'da', 'Pinhal', 'em', 'São', 'Paulo', '.']
Encoded: [13964, 3296, 184, 3, 158, 6328, 747, 20, 391, 10, 13965, 11, 30, 51, 2]


In [ ]:
tag_tokenizer = Tokenizer(filters=None)
tag_tokenizer.fit_on_texts(Y)
Y_encoded = tag_tokenizer.texts_to_sequences(Y)

print(f"Original: {Y[0]}")
print(f"Encoded: {Y_encoded[0]}")

Original: ['N', 'V', 'N', 'PREP', 'CUR', 'NUM', 'N', 'PREP+ART', 'N', 'PREP+ART', 'NPROP', 'PREP', 'NPROP', 'NPROP', 'PU']
Encoded: [1, 3, 1, 5, 19, 12, 1, 7, 1, 7, 4, 5, 4, 4, 2]


Como a arquitetura LSTM exige que todas as entradas possuam o mesmo tamanho, as sentenças devem ser preenchidas. Entretanto, o tamanho da sentença final, após ser realizado o "padding", é arbitrário. No trecho abaixo, podemos ter uma noção da distribuição de tamanhos das sentenças no dataset. 75% das sentenças possuem um tamanho menor ou igual a 25. Para ter uma boa margem de segurança, o tamanho final das sentenças que escolhi será de 100. Entretanto, isso implica que as senteças que tiverem mais do que 100 palavras serão truncadas, o que ocorre raramente no dataset.

In [ ]:
lengths = [len(sentence) for sentence in X_encoded]

df = pd.DataFrame(lengths)
df.describe()

,0
count,49932.000000
mean,18.940779
std,12.070051
min,1.000000
25%,10.000000
50%,17.000000
75%,25.000000
max,248.000000


Padding é feito à esquerda e a truncagem é feita à direita.

In [ ]:
MAX_SEQ_LENGTH = 100

X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

In [ ]:
print(f"X padded: {X_padded[0]}")
print(f"Y padded: {Y_padded[0]}")

X padded: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0 13964  3296   184     3   158  6328   747    20   391    10 13965
    11    30    51     2]
Y padded: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  1  3  1  5 19 12  1  7  1  7  4
  5  4  4  2]


In [ ]:
X, Y = X_padded, Y_padded
print(f"X shape after padding:{X.shape}")
print(f"Y shape after padding:{Y.shape}")

X shape after padding:(49932, 100)
Y shape after padding:(49932, 100)


Agora, como queremos um vetor de probabilidades para cada uma das tags como saída para cada palavra da sentença, transformamos Y em uma variável categórica utilizando o "one-hot-enconding".

In [ ]:
Y = to_categorical(Y)
Y.shape

(49932, 100, 27)

# Treinamento
Primeiro, separo o dataset nos datasets de treino, validação e teste. Como essa separação já vem pronta, basta recuperar utilizando os índices correspondentes aos tamanhos dos subconjuntos.

In [ ]:
X_train, Y_train = X[:train_len], Y[:train_len]
X_validation, Y_validation = X[train_len:(train_len+dev_len)], Y[train_len:(train_len+dev_len)]
X_test, Y_test = X[(train_len+dev_len):], Y[(train_len+dev_len):]

print(f"Train:\nX shape:{X_train.shape} Y shape:{Y_train.shape}")
print(f"Validation:\nX shape:{X_validation.shape} Y shape:{Y_validation.shape}")
print(f"Test:\nX shape:{X_test.shape} Y shape:{Y_test.shape}")

Train:
X shape:(37948, 100) Y shape:(37948, 100, 27)
Validation:
X shape:(1997, 100) Y shape:(1997, 100, 27)
Test:
X shape:(9987, 100) Y shape:(9987, 100, 27)


Agora eu construo o modelo neural propriamente dito. A primeira camada da arquitetura é uma camada de Embedding, responsável por aprender a representação neural das palavras, que estão codificadas como números inteiros. O tamanho dos embeddings escolhido foi 300.

A próxima camada da arquitetura é a de LSTM, com o tamanho da camada oculta de 64. Como return_sequences é igual a True, o output dessa camada vai ser um vetor contendo o output da célula LSTM para cada uma das palavras, logo terá um shape de (100, 64).

A última camada da arquitetura é uma de TimeDistribuited. Essa camada apenas aplica uma camada Densa (fully connected) para cada iteração do timestep, com uma ativação de softmax. De outra forma, temos para cada palavra, uma camada densa que irá fornecer as probabilidades daquela palavra de pertencer a cada uma das tags.

In [ ]:
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1 # precisa adicionar + 1 para englobar o "0", que representa o padding
NUM_CLASSES = NUM_TAGS + 1 # também precisa adicionar + 1 para englobar o "0", que representa o padding
EMBEDDING_SIZE = 300

lstm_model = Sequential()

# Layer de Embedding
lstm_model.add(Embedding(input_dim=VOCABULARY_SIZE, output_dim=EMBEDDING_SIZE, input_length=MAX_SEQ_LENGTH, trainable=True))

# Layer de LSTM, com tamanho do hidden state igual a 64. Como return_sequences é True, o output é um vetor com as saídas para cada palavra na setença.
lstm_model.add(LSTM(64, return_sequences=True))

# Layer de TimeDistribuited. Para cada palavra, temos um softmax para obter a distribuição de probabilidade para todas as tags.
lstm_model.add(TimeDistributed(Dense(NUM_CLASSES, activation='softmax')))

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          16270800  
                                                                 
 lstm (LSTM)                 (None, 100, 64)           93440     
                                                                 
 time_distributed (TimeDistr  (None, 100, 27)          1755      
 ibuted)                                                         
                                                                 
Total params: 16,365,995
Trainable params: 16,365,995
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_training = lstm_model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_validation, Y_validation))

Epoch 1/10
297/297 [==============================] - 120s 398ms/step - loss: 0.5331 - acc: 0.8784 - val_loss: 0.1823 - val_acc: 0.9536
Epoch 2/10
297/297 [==============================] - 117s 393ms/step - loss: 0.1110 - acc: 0.9728 - val_loss: 0.0773 - val_acc: 0.9806
Epoch 3/10
297/297 [==============================] - 119s 401ms/step - loss: 0.0577 - acc: 0.9856 - val_loss: 0.0576 - val_acc: 0.9840
Epoch 4/10
297/297 [==============================] - 115s 386ms/step - loss: 0.0429 - acc: 0.9889 - val_loss: 0.0513 - val_acc: 0.9857
Epoch 5/10
297/297 [==============================] - 114s 383ms/step - loss: 0.0360 - acc: 0.9903 - val_loss: 0.0485 - val_acc: 0.9861
Epoch 6/10
297/297 [==============================] - 115s 388ms/step - loss: 0.0316 - acc: 0.9913 - val_loss: 0.0469 - val_acc: 0.9864
Epoch 7/10
297/297 [==============================] - 114s 382ms/step - loss: 0.0283 - acc: 0.9920 - val_loss: 0.0466 - val_acc: 0.9865
Epoch 8/10
297/297 [============================

# Avaliação/Resultados
Podemos ver inicialmente que a acurácia no dataset de teste é excelente, sendo em torno de 98%.

In [ ]:
loss, accuracy = lstm_model.evaluate(X_test, Y_test, verbose = 1)
print(f"Loss: {loss}\nAccuracy: {accuracy}")

313/313 [==============================] - 7s 21ms/step - loss: 0.0526 - acc: 0.9852
Loss: 0.0526154525578022
Accuracy: 0.9851827621459961


Para obter a acurácia por classe gramatical, primeiro faço as predições no dataset de teste e "excluo" a primeira dimensão da saída ao combinar as duas primeiras dimensões. Dessa forma, não há mais separação em sentenças. Depois, obtenho a classe predita para cada palavra utlizando a função np.argmax, que retorna o índice do maior valor de um vetor. Ao final, tenho um vetor "y_hat", que são as predições do modelo, e o vetor "y", contendo a saída correta.

In [ ]:
predictions = lstm_model.predict(X_test)
predictions.shape

(9987, 100, 27)

In [ ]:
nrows, seq_len, num_class = predictions.shape

flat_X = np.reshape(predictions, (nrows * seq_len, num_class))
y_hat = np.argmax(flat_X, axis=1)

flat_Y = np.reshape(Y_test, (nrows * seq_len, num_class))
y = np.argmax(flat_Y, axis=1)

print(f"flat_X shape: {flat_X.shape} flat_Y shape:{flat_Y.shape}")
print(f"label: {y}\nprediction:{y_hat}")

flat_X shape: (998700, 27) flat_Y shape:(998700, 27)
label: [0 0 0 ... 5 1 2]
prediction:[0 0 0 ... 5 1 2]


In [ ]:
pred_df = pd.DataFrame({'y': y, 'y_hat': y_hat}).query('y != 0')
pred_df['hit'] = (pred_df['y'] == pred_df['y_hat']).astype(int)

percentage = pred_df.groupby('y').count()['y_hat'] / len(pred_df)

results = pred_df[['y', 'hit']].groupby('y').mean().join(percentage).sort_values(by='y_hat', ascending=False)
results.columns = ['accuracy', 'presence_in_data']

tag_name_map = {value : key for (key, value) in tag_tokenizer.word_index.items()}
tag_name_map = pd.DataFrame.from_dict(tag_name_map, orient='index', columns=['tag_name'])

results = results.join(tag_name_map)
results

,accuracy,presence_in_data,tag_name
y,,,
1,0.937813,0.204876,n
2,0.999926,0.150827,pu
3,0.932497,0.110501,v
5,0.947083,0.094078,prep
4,0.771703,0.089414,nprop
6,0.973263,0.070533,art
7,0.988247,0.057304,prep+art
8,0.873200,0.047937,adj
9,0.838484,0.030510,adv


Aqui podemos ver que para as 10 tags mais presentes no dataset de teste, a acurácia do modelo foi excelente. Na maioria, a acurácia esteve acima de 90%, sendo a menor delas 80% para a classe de "Nome Próprio".

Para obter informações mais detalhadas, como a acurácia e a proporção nos dados exata, basta passar o cursor do mouse em cima das barras do gráfico.

In [ ]:
results = results.sort_values(by='presence_in_data', ascending=False)
fig = px.bar(results.iloc[:10], 
            x='tag_name',
            y='accuracy',
            text=[f'Proporção: {(i*100):.2f}%' for i in results.iloc[:10]['presence_in_data']],
            hover_data=['presence_in_data'],
            title='Acurácia das 10 primeiras tags ordenadas pela proporção nos dados')
fig.show()

Desconsiderando a proporção das tags no dataset por enquanto, novamente temos um resultado excelente para as 10 tags com a melhor acurácia, sendo a menor delas 96%. Essas tags são, em ordem de acurácia: "Pontuação", "Moeda Corrente", "Preposição + Pronome Adjetivo", "Preposição + Artigo", "Artigo", "Conjunção Coordenativa", "Preposição", "Nome", "Pronome Pessoal" e "Verbo".

In [ ]:
results = results.sort_values(by='accuracy', ascending=False)
fig = px.bar(results.iloc[:10], 
            x='tag_name',
            y='accuracy',
            text=[f'Proporção: {(i*100):.2f}%' for i in results.iloc[:10]['presence_in_data']], 
            hover_data=['presence_in_data'],
            title='10 melhores acurácias por tag')
fig.show()

Para as 10 piores acurácias por tag, temos em ordem crescente de acurácia: "Preposição + Advérbio", "Preposição + Pronome Conectivo Subordinativo", "Interjeição", "Preposição + Pronome Substantivo", "Pronome Substantivo", "Advérbio Conectivo Subordinativo", "Conjunção Subordinativa", "Palavra Denotativa", "Nome Próprio" e "Numeral".

Podemos ver que para algumas tags temos uma acurácia bem ruim, chegando a 0%, enquanto outras tem entre 1% e 50%, embora a 10ª tag ainda possua uma acurácia de 83%, o que demonstra que são poucos os casos em que a performance do modelo é insatisfatória. Entretanto, uma coisa que podemos notar nessas classes é que elas ocorrem raramente no dataset, chegando em sua maioria a menos de 1% de presença do dataset de teste. Isso pode explicar o desempenho ruim dessas tags, já que não houveram exemplos suficientes para o modelo aprender a prever corretamente essas classes.



In [ ]:
results = results.sort_values(by='accuracy')
fig = px.bar(results.iloc[:10], 
            x='tag_name',
            y='accuracy',
            text=[f'Proporção: {(i*100):.2f}%' for i in results.iloc[:10]['presence_in_data']], 
            hover_data=['presence_in_data'],
            title='10 piores acurácias por tag')
fig.show()